In [1]:

import numpy as np
from absl.testing import parameterized

from tensorflow.python.keras import keras_parameterized
from tensorflow.keras.backend import floatx
import tensorflow as tf

from autopycoin.dataset.generator import WindowGenerator

from autopycoin.models import PoolNBEATS

from autopycoin.utils import layer_test, check_attributes
from autopycoin.losses import QuantileLossError
from autopycoin.layers.nbeats_layers import GenericBlock, TrendBlock, SeasonalityBlock
from autopycoin.models import (
    create_interpretable_nbeats,
    create_generic_nbeats,
    NBEATS,
    Stack,
)
from autopycoin.data import random_ts


data = random_ts(n_steps=30, n_variables=2)

w = WindowGenerator(
        input_width=6,
        label_width=3,
        shift=3,
        valid_size=10,
        test_size=0,
        flat=False)
w.from_array(data, input_columns=[0, 1], label_columns=[0, 1])

model = [create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        ),create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        )]

model = PoolNBEATS(
    n_models=10,
    nbeats_models=model,
    losses=["mse", "mae", "mape"],
    fn_agg=tf.reduce_mean,
)

model.compile(
    tf.keras.optimizers.Adam(
        learning_rate=0.02,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam",
    ),
    loss=model.get_pool_losses(),
    metrics="mae",
)

# Issue 13
model.fit(w.train, epochs=10, validation_data=w.valid)

AttributeError: __name__

In [8]:
type(create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        ))

autopycoin.models.nbeats.NBEATS